## Introduction:

While working on becoming a full time web developer, I decided take a part-time job working for a local non-profit. One of my tasks involved logging into a third party website, downloading a CSV file, formatting it in Excel and then printing it for use in a class that occurred twice a week.

I soon realized that this was a perfect opportunity to put my skills as a programmer to use. So I set out to automate this task. Here are the steps I took to make this happen.




In [1]:
#Import used:

from dotenv import load_dotenv
load_dotenv()

import os

import requests

from requests.auth import HTTPBasicAuth;

from bs4 import BeautifulSoup;

import re;

import datetime

import csv

import pandas as pd

import xlsxwriter

import yagmail

## Step One: Get the class ID

When first logging into the third party site with the data I needed, I was met with a table with columns that included the date of the class and its id. Clicking on the link for any particular class, I noted that the URL had two parameters: the date of the class and its ID. At first, I thought that simply passing the date might be enough. After all, there was only one class occurring on any particular date. I tested passing just the date and not the class id. I got back a table of data but it was the wrong data. So, as a first step in the project, I needed to get the id of the class.

In [2]:
#Using a context manager, I open the first session to the third party website to get table
#with class ids and dates: 

#Environment vars I need for this:
site = os.getenv("ADMIN_SITE")
user = os.getenv("ADMIN_USER")
pswd = os.getenv("ADMIN_PSWD")

with requests.Session() as s:
    resp = s.get(site, auth=HTTPBasicAuth(user, pswd), timeout=10)
    
    if resp.status_code == 200:
        class_list = resp.content;
        print("Connection successful") #I add this line to show connection was successful.
        
    else:
        print("Connection could not be establised. Status code: {}".format(resp.status_code))

Connection successful


## Part 2: Get class ids and dates

Having successfully connected to the site, I now had access to the HTML that included the table with class dates and ids. As a next step, I scraped that contents of the table and placed them in a Python dictionary for later use.


In [3]:
soup = BeautifulSoup(class_list, 'html.parser')

links = soup.find_all('a')

#The links I parsed were returned with the class id and date as attributes. So I used regular 
#expressions to pull out the content I needed with the following regex:

id_regex = re.compile("(?<=class_id=)\d{3,}")

date_regex = re.compile("(?<=class_date=)\d{4}-\d{2}-\d{2}")

#Empty lists to hold id and class date data:

ids = []

dates = []

#Loop through links to get data I need and append the result to the lists above:


for item in links:

    id_matches = re.findall(id_regex, str(item))

    date_matches = re.findall(date_regex, str(item))

    if len(id_matches) > 0:

        ids.append(id_matches[0])

    if len(date_matches) > 0:

        dates.append(date_matches[0])

#Finally, I zip the id and dates into a dictionary.
dateId = dict(zip(ids, dates))


#Here is the dateId data that resulted from all this:

print(dateId)

{'623': '2019-01-01', '599': '2019-06-01', '600': '2019-06-02', '601': '2019-06-03', '602': '2019-06-04', '603': '2019-06-08', '604': '2019-06-09', '605': '2019-06-10', '606': '2019-06-11', '607': '2019-06-15', '608': '2019-06-16', '609': '2019-06-17', '610': '2019-06-18', '611': '2019-06-22', '612': '2019-06-23', '613': '2019-06-24', '614': '2019-06-25', '615': '2019-06-29', '616': '2019-06-30', '617': '2019-07-01', '618': '2019-07-02', '619': '2019-07-06', '620': '2019-07-07', '621': '2019-07-08', '622': '2019-07-09', '624': '2019-07-13', '625': '2019-07-14', '626': '2019-07-15', '627': '2019-07-16', '628': '2019-07-20', '629': '2019-07-21', '630': '2019-07-22', '632': '2019-07-23', '633': '2019-07-27', '634': '2019-07-28', '635': '2019-07-29', '636': '2019-07-30', '637': '2019-08-03', '638': '2019-08-04', '639': '2019-08-05', '640': '2019-08-06', '641': '2019-08-10', '642': '2019-08-11', '643': '2019-08-12', '644': '2019-08-13', '645': '2019-08-17', '646': '2019-08-18', '647': '2019

## Part 3: Is there a class today?

Now that I have the class dates and their respective ids, I could now test if there was a class on any particular date and, if there was, gather the data I would need for my excel spreadsheet.

In [4]:
#First, I get today's date:

today = datetime.datetime.today().strftime('%Y-%m-%d')

#Next I check if there is a date in the dictionary above that matches today's date. If so, I
#grab the class date and id:

for k, v in dateId.items():

    if v == today:

        classId = k;

        classDate = v;     
        
#Using an environment variable to get part of the url I'll need:
classLink = os.getenv("CLASS_LINK")

#Now if there is a class date that matches today's date, I use it to create the url I'll need
#for the 2nd HTTP request:

if classDate == today:

    newUrl = "{}class_id={}&class_date={}".format(classLink, classId, classDate)
    
else:

    print("No class today")


In [6]:
#Next I open a new HTTP request using the newUrl created because we did not get a message
#saying there was no class today:

if len(newUrl) != 0:

    with requests.Session() as s2:

        resp2 = s2.get(newUrl,auth=HTTPBasicAuth(user, pswd), timeout=10)
        
#Now I parse the resp2 result:

table_soup = BeautifulSoup(resp2.content, "html.parser")

#And extract the table data, which I found located in a table with the class of "body-text"

class_register = table_soup.find("table", {"class" : "bodytext"} )

## Part 4: Extract data from HTML table and initial formatting

In this next step, I want to extract the data I need from the HTML tags. Going forward, I will 
ask the reader to assume that I am succeeding in these steps as I do not want to publish data without permission.

In [8]:
#pull out all table rows:

table_rows = class_register.find_all('tr')

#creating an empty data list to hold the data:

data = [];

#looping through the table rows to get the data and populate it into the data array:

for tr in table_rows:

    td = tr.find_all('td')

    row = [i.text for i in td]

    data.append(row)
    
#to make it easier to visualize and work with the resulting data, I used pandas:

dat = pd.DataFrame(data[1:], columns=data[0])

#Removing some unneeded columns:

dat.drop(['Request Date', 'SSN', 'Attended', 'DL Number', 
          'TLC Number', 'Car Number', 'Class ID'], axis=1, inplace=True)

#There was one more column that I wanted to remove:

d = dat.iloc[:, 1:]

#I need an empty Signature column to gather student signatures later

blank = pd.DataFrame(columns=['Signature'])

d = pd.concat([d, blank], axis=1)

d.fillna(" ", inplace=True)

## Step 5: Complete formatting of the data and store to excel file

In [10]:
#Helper function to remove whitespaces and capitalize certain columns:

def stripAndCap(col):

    d[col] = d[col].str.strip().str.capitalize()
    
cap_cols = ['First Name', 'Last Name', 'City',  'Company Base']

[stripAndCap(i) for i in cap_cols]

#Now a series of data formatting with the help of pandas, list comprehensions, regex, etc.

d['State'] = d['State'].str.strip().str.upper()
d['Email'] = d['Email'].str.strip().str.lower()
d['Cell Number'] = d['Cell Number'].str.replace(r'[^0-9]', '').str.replace(r'^1', '')
d['Cell Number'] = ["-".join(re.match("(\d{3})(\d{3})(\d{4})", i).groups()) 
                    for i in d['Cell Number']]
d['Company Base'] = d['Company Base'].str.strip().str.replace(r'(?<=\w)[-|,]\S+', '')

for i in d['Address'].str.strip():

    for j in i.split():

        ''.join(j.capitalize())

d['Address'] = [' '.join([j.capitalize() for j in i.split()]) for i in d['Address'].str.strip()]

#Sort the values on Last Name variable:

d.sort_values(by=['Last Name'], inplace=True)

#Realign columns in order I need them:

d = d[['Last Name', 'First Name', 'Address', 'City', 'State', 'Zip',

         'Company Base', 'Cell Number', 'Email', 'Signature']]



## Part 6: Importing data into excel and prepping data for printing

In this step, I make use of the xlsxwriter package to prepare the data to be print ready for the end user.

In [12]:
#Note the name of the written file: "DDC_[today's date].xlsx"

with pd.ExcelWriter(f'DDC_{today}.xlsx', engine='xlsxwriter') as writer:

    d.to_excel(writer, header=True, index=False)
    
    workbook = writer.book

    worksheet = writer.sheets['Sheet1']
    
    worksheet.set_header('&C&24&"Bold"Class Register &D')
    
    worksheet.set_default_row(60, hide_unused_rows=True)
    
    #Setting the formats of each of the column headers.
    
    header_format = workbook.add_format({

                                            'bold': 1,

                                            'align': 'center',

                                            'font_size': 16,

                                            'valign': 'vcenter',

                                            'bg_color': "#F8F8F8",

                                            'bottom' : 6,

                                            'left': 1,

                                            'right': 1

                                            })
    
    for col, val in enumerate(d.columns):

        worksheet.write(0, col, val, header_format)
        
    #Adding some default column formatting
        
    defaultColFormat = workbook.add_format({'font_size': 15,

                                                'align': 'center',

                                                'text_wrap': 1

                                                })



    worksheet.set_column(0, 10, 16, defaultColFormat)
    
    #Adjusting the widths of certain columns:
    
    worksheet.set_column('C:C', 36)

    worksheet.set_column('D:D', 14)

    worksheet.set_column('E:E', 6)

    worksheet.set_column('F:F', 8)

    worksheet.set_column('G:G', 18)

    worksheet.set_column('I:I', 36)

    worksheet.set_column('J:J', 48)
    
    #Setting page layout and printing options:
    
    
    worksheet.set_landscape()

    worksheet.center_horizontally()

    worksheet.set_margins(left=0, right=0, top=0.6, bottom=0.1)

    worksheet.hide_gridlines(0)

    worksheet.fit_to_pages(1, 2)

    worksheet.repeat_rows(0)

    #Finally, I save the 'writer' context manager
    
    writer.save()




In [13]:
#Note that the DDC excel file has been successfully created in the project directory 
#(DDC_2019-11-11.xlsx):
%ls

DDC.py                    WebScrapingProject.ipynb
DDC_2019-11-11.xlsx       oauth2.json


## Step 7: Setting up the emailing of the excel file to office admin email

For this step of the process, which I won't include the code for, I wanted to email the xlsx file created by my program to the office admin email. Here is a general overview of the steps I took in lieu of code:

- Imported yagmail
- Visited google developer link to enable the gmail api.
- In the gmail api, I created a new project and obtained the client id and secret.
- I ran 'yag = yagmail.SMTP('my email', oauth2_file="oauth2.json"). The oauth2 file was left blank at first.
- Running the above code, I was prompted to enter my email, client id and client secret. This then led to a link I had to visit in gmail to get a token. I then copied and pasted the token into the final prompt. At this point, I was able to connect to gmail API with OAuth2.
- After successfully testing this process, I set my contents variable equal to the name of the xlsx file. It is important to include the xlsx part of the file to ensure correct processing.
- Then, I ran yag.send(to=[the admin email], contents = contents)

The above steps resulted in a successful transmission of my excel file to the admin email

## Step 8: Automating the running of this code

As a final step, I set up a bash script to run the python code described here. Then, I opened a new crontab where I schedule the bash script to run every morning at 7am.

## Final Thoughts:

Even though I successfully automated this process, I realize that some of my code here is, perhaps, not 'pythonic'. Speed was not a concern here. The point was to successfully get from point A to point B, which I did. 